In [1]:
%matplotlib notebook
from __future__ import division

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from matplotlib.pyplot import plot, ylabel, xlabel, yscale, xscale, legend, subplots
from theano import function
import numpy as np
import gzip
import cPickle
from scipy.optimize import minimize
from climin.util import optimizer
from itertools import repeat, cycle, islice, izip
inf = float("inf")

gnumpy: failed to import cudamat. Using npmat instead. No GPU will be used.


In [3]:
from breze.learn.data import one_hot
from breze.learn.base import cast_array_to_local_type
from schlichtanders.myfunctools import compose
from schlichtanders.myoptimizers import batch
from schlichtanders.mygenerators import eatN, chunk, every
from schlichtanders.myplot import add_val, add_point

In [4]:
from theano_models import (as_tensor_variable, total_size, Merge, clone, clone_all,
                           reparameterize, squareplus, squareplus_inv, pmerge_key_reparam,
                           InvertibleModel)
from theano_models.visualization import d3viz
from IPython.display import IFrame
import theano_models.deterministic_models as dm
import theano_models.probabilistic_models as pm
import theano_models.postmaps as post
from theano_models.composing import normalizing_flow, variational_bayes

In [5]:
from theano_models.util.theano_helpers import get_dependencies, sort_dependent_last

In [6]:
from theano.printing import debugprint

# Data

In [7]:
datafile = '../data/mnist.pkl.gz'
# Load data.        a                                                                                           

with gzip.open(datafile,'rb') as f:                                                                        
    train_set, val_set, test_set = cPickle.load(f)                                                       

X, Z = train_set                                                                                               
VX, VZ = val_set
TX, TZ = test_set

Z = one_hot(Z, 10)
VZ = one_hot(VZ, 10)
TZ = one_hot(TZ, 10)

image_dims = 28, 28

X, Z, VX, VZ, TX, TZ = [cast_array_to_local_type(i) for i in (X, Z, VX,VZ, TX, TZ)]
map(np.shape, [X, Z, VX, VZ, TX, TZ])

[(50000, 784),
 (50000, 10),
 (10000, 784),
 (10000, 10),
 (10000, 784),
 (10000, 10)]

# Model

## data modelling

In [8]:
predictor = dm.Mlp(output_size=10, output_transfer="softmax", hidden_sizes=[200]*2, hidden_transfers=["rectifier"]*2)
post.flatten_parameters(predictor)

noise = pm.DiagGaussianNoise(predictor)

targets = Merge(noise, inputs=predictor['inputs'], to_be_randomized=predictor['parameters_flat'])
targets

{ 'inputs': [<TensorType(float64, vector)>],
  'logP': <FunctionWrapper at 0x7fa483cd2910 for function at 0x7fa483d2cd70>,
  'outputs': Elemwise{add,no_inplace}.0,
  'parameters': [],
  'parameters_positive': [var],
  'to_be_randomized': weights_copy:bias_copy:weights_copy:bias_copy:weights_copy:bias_copy}

## parameter modelling

In [9]:
params_base = pm.DiagGauss(output_size=total_size(targets['to_be_randomized']))  # if you want to use size directly, CAUTION, you need to copy before!
# params_base.map('parameters_positive', reparameterize_map(squareplus, squareplus_inv), 'parameters')

normflows = [dm.PlanarTransform() for _ in range(2)]
params_cur = params_base
for transform in normflows:
    params_cur = normalizing_flow(transform, params_cur)  # returns transform, however with adapted logP    

params = Merge(params_cur, params_base, *normflows[:-1])  # last one is params_cur
params

{ 'inputs': [],
  'inverse_inputs': [inverse_inputs],
  'inverse_outputs': inverse_outputs,
  'logP': <FunctionWrapper at 0x7fa483ca6980 for function at 0x7fa483ca5a28>,
  'norm_det': Elemwise{add,no_inplace}.0,
  'outputs': f_outputs,
  'parameters': [b, w, _u, mean, b, w, _u],
  'parameters_positive': [var]}

## bayes

In [10]:
prior = pm.DiagGauss(targets['to_be_randomized'].size)
prior.mean.name = "mean_prior"

variational_bayes(targets, 'to_be_randomized', params, priors=prior)

model = Merge(targets, params, prior, parameters=pmerge_key_reparam(squareplus, squareplus_inv))
model

No substitution as `self[logP]` is not a theano.gof.Variable. Key is simply replaced.


{ 'inputs': [<TensorType(float64, vector)>],
  'kl_prior': Elemwise{sub,no_inplace}.0,
  'logP': <FunctionWrapper at 0x7fa483c6e1a0 for function at 0x7fa483ca5410>,
  'loglikelihood': <FunctionWrapper at 0x7fa483cd2910 for function at 0x7fa483d2cd70>,
  'n_data': <TensorType(int64, scalar)>,
  'outputs': Elemwise{add,no_inplace}.0,
  'parameters': [ b,
                  w,
                  _u,
                  mean,
                  b,
                  w,
                  _u,
                  mean_prior,
                  var_squareplus,
                  var_squareplus,
                  var_squareplus],
  'parameters_positive': [var_reparam, var_reparam, var_reparam],
  'to_be_randomized': inverse_outputs}

# Visualizing model

In [11]:
d3viz(model, [targets, noise, predictor, params] + normflows + [params_base], 'tmp/visualization.html')
IFrame('tmp/visualization.html', width=700, height=500)

model { 'inputs': [<TensorType(float64, vector)>],
  'kl_prior': Elemwise{sub,no_inplace}.0,
  'logP': <FunctionWrapper at 0x7fa483c6e1a0 for function at 0x7fa483ca5410>,
  'loglikelihood': <FunctionWrapper at 0x7fa483cd2910 for function at 0x7fa483d2cd70>,
  'n_data': <TensorType(int64, scalar)>,
  'outputs': Elemwise{add,no_inplace}.0,
  'parameters': [],
  'parameters_positive': [var_reparam],
  'to_be_randomized': inverse_outputs} defaultdict(<type 'list'>, {'ext_outputs': [(Elemwise{add,no_inplace}.0, 'n1')], 'ext_inputs': ['n3']})
model { 'inputs': [Reshape{1}.0],
  'logP': <FunctionWrapper at 0x7fa483cd2910 for function at 0x7fa483d2cd70>,
  'outputs': Elemwise{add,no_inplace}.0,
  'parameters': [],
  'parameters_positive': [var_reparam]} defaultdict(<type 'list'>, {'ext_outputs': [(Elemwise{add,no_inplace}.0, 'n21')], 'ext_inputs': ['n24']})
model { 'inputs': [<TensorType(float64, vector)>],
  'outputs': Reshape{1}.0,
  'parameters': [ weights_flat,
                  bias_flat,

TypeError: make_nested_model() takes exactly 7 arguments (6 given)

# Optimizer

In [ ]:
InvertibleModel.reduce_all_identities()

In [ ]:
postmap = compose(post.flat_numericalize_postmap, post.variational_postmap, post.flatten_parameters)
postmap_kwargs = {
    'wrapper': batch,
    'initial_inputs': [X[0]],
    'adapt_init_params': lambda ps: ps + np.random.normal(size=ps.size) * 0.01
}
optimizer_kwargs = postmap(model, **postmap_kwargs)
post.climin_postmap(optimizer_kwargs)

Climin wants an iterator of (args, kwarsg) as keyword argument "args" (to be passed to the loss function). Concretley, we use an infinite iterator over minibatches.

In [ ]:
batch_size = 20
climin_args = izip(izip(chunk(batch_size, cycle(Z)), chunk(batch_size, cycle(X))), repeat({}))

In [ ]:
opt = optimizer(
    identifier = "adadelta",
    args=climin_args, # repeat(((Z,X),{})),
    **post.climin_postmap(optimizer_kwargs)
)

# Visualized Fit

In [ ]:
line_train, = plot([], [], 'go-', label="average training loss")
line_curr_val, = plot([],[], 'bo:', label="avrg current validation loss")
line_best_val, = plot([], [], 'ko-', label="avrg best validation loss")
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
yscale('log')
ylabel("validation loss")
xlabel("#iteration")
legend(loc='lower left', fancybox=True, framealpha=0.5)

In [ ]:
best_val_loss = inf
best_wrt = None
val_size = batch_size

n_whole_data = X.shape[0] // batch_size

for info in every(n_whole_data, opt):
    # collect and visualize validation loss for choosing the best model
    val_loss = optimizer_kwargs['num_loss'](opt.wrt, VZ[:val_size], VX[:val_size])/val_size
    if val_loss < best_val_loss:
        best_wrt = opt.wrt
        best_val_loss = val_loss
        add_point(line_best_val, info['n_iter'], val_loss)
    add_point(line_curr_val, info['n_iter'], val_loss)
    
    # visualize training loss for comparison:
    try:
        training_loss = info['loss'] / len(Z)  # TODO normalization needed?
    except KeyError:
        training_loss = optimizer_kwargs['num_loss'](opt.wrt, Z, X)/len(Z)
    add_point(line_train, info['n_iter'], training_loss)

# Performance

TODO: average over predictions

In [ ]:
print best_val_loss
mlp['parameters_flat'] = best_wrt

predict = mlp.function()
predict(X[0]), Z[0]

In [ ]:
PX = np.apply_along_axis(predict, 1, X)
PVX = np.apply_along_axis(predict, 1, VX)
PTX = np.apply_along_axis(predict, 1, TX)

In [ ]:
print 'incorrect samples train/val/test:  %i/%i/%i' % (
    (PX[:, :10].argmax(1) != Z.argmax(1)).sum(),
    (PVX[:, :10].argmax(1) != VZ.argmax(1)).sum(),
    (PTX[:, :10].argmax(1) != TZ.argmax(1)).sum())

print 'error rate train/val/test:  %g/%g/%g' % (
    (PX[:, :10].argmax(1) != Z.argmax(1)).mean(),
    (PVX[:, :10].argmax(1) != VZ.argmax(1)).mean(),
    (PTX[:, :10].argmax(1) != TZ.argmax(1)).mean())